# Inversion benchmark - Hofmeister 1999

The full Hofmeister 1999 expression contains many terms that describe the rheology of the lithosphere. We derive the formula with respect to each of these properties using `sympy`.

The most important terms we consider here are temperature, $T$, and initial thermal conductivity, $k_0$.

In [ ]:
import numpy as np

In [ ]:
import sympy
sympy.init_printing()

a = 2.85e-5
b = 1.008e-8
c = -0.384
d = 0.0

e = 0.0175
f = 1.0374e-4
g = 2.245/1e7
h = 3.407/1e11
i = 3350.0*1e-9
j = 88804.0
k = 3.3557e-3
l = 2.35e12
m = 1.0/3

K_0p, g_av, zL, gmma_T, K_T = sympy.symbols('K_p g_mu z_L gamma K_T')
a, b, c, d, k0, T = sympy.symbols('a b c d k_0 T')
e,f,g,h,i,j,k,l,m = sympy.symbols('e,f,g,h,i,j,k,l,m')

k_con  = k0*(298.0/T)**0.45
k_rad  = e - f*T + g*T**2 - h*T**3
k_pres = 1.0 + (K_0p*g_av*zL*i)/K_T
k_exp  = sympy.exp(-(a*(T - 298.0) + b*0.5*(T**2 - j) + \
                 c*(k - 1.0/T) + d*(T**5 - l)/5.0)* \
                 (gmma_T*4 + m))
k_new = k_con*k_exp*k_pres + k_rad

In [ ]:
k_new

In [ ]:
k_new.diff(k0)

In [ ]:
0.45*298**0.45

In [ ]:
k_new.diff(T)

In [ ]:
def forward_model(x):
    k0, T = np.array_split(x, 2)
    
    a = 2.85e-5
    b = 1.008e-8
    c = -0.384
    d = 0.0

    k_con  = k0*(298.0/T)**0.45
    k_rad  = 0.0175 - 1.0374e-4*T + 2.245*T**2/1e7 - 3.407*T**3/1e11
    k_pres = 1.0 + (K_0p*g_av*3350.0*np.abs(coords)*1e-9)/K_T
    k_exp  = np.exp(-(a*(T - 298.0) + b*0.5*(T**2 - 88804.0) + \
                     c*(3.3557e-3 - 1.0/T) + d*(T**5 - 2.35e12)/5.0)* \
                     (gmma_T*4 + 1.0/3))
    k_new = k_con*k_exp*k_pres + k_rad
    return k_new.sum()

def tangent_linear(x, dx):
    k0, T = np.array_split(x, 2)
    dk0, dT = np.array_split(dx, 2)
    
    a = 2.85e-5
    b = 1.008e-8
    c = -0.384
    d = 0.0

    k_con  = k0*(298.0/T)**0.45
    k_rad  = 0.0175 - 1.0374e-4*T + 2.245*T**2/1e7 - 3.407*T**3/1e11
    k_pres = 1.0 + (K_0p*g_av*3350.0*np.abs(coords)*1e-9)/K_T
    k_exp  = np.exp(-(a*(T - 298.0) + b*0.5*(T**2 - 88804.0) + \
                     c*(3.3557e-3 - 1.0/T) + d*(T**5 - 2.35e12)/5.0)* \
                     (gmma_T*4 + 1.0/3))
    
    dkdk0 = k_pres*k_exp*(298.0/T)**0.45
    dkdT  = -3.0*3.407e-11*T**2 + 2.0*T*2.245e-7 - 1.0374e-4 + \
            k_pres*k_con*(4.0*gmma_T + 1.0/3)*(-T**4*d - T*b - a - c/T**2) - \
            k_pres*0.45/T*k_con*k_exp
    dk = dkdk0*dk0 + dkdT*dT
    return dk.sum()

def adjoint_model(x):
    k0, T = np.array_split(x, 2)
    dk0, dT = np.array_split(dx, 2)
    
    a = 2.85e-5
    b = 1.008e-8
    c = -0.384
    d = 0.0

    k_con  = k0*(298.0/T)**0.45
    k_rad  = 0.0175 - 1.0374e-4*T + 2.245*T**2/1e7 - 3.407*T**3/1e11
    k_pres = 1.0 + (K_0p*g_av*3350.0*np.abs(coords)*1e-9)/K_T
    k_exp  = np.exp(-(a*(T - 298.0) + b*0.5*(T**2 - 88804.0) + \
                     c*(3.3557e-3 - 1.0/T) + d*(T**5 - 2.35e12)/5.0)* \
                     (gmma_T*4 + 1.0/3))
    
    dkdk0 = k_pres*k_exp*(298.0/T)**0.45
    dkdT  = -3.0*3.407e-11*T**2 + 2.0*T*2.245e-7 - 1.0374e-4 + \
            k_pres*k_con*(4.0*gmma_T + 1.0/3)*(-T**4*d - T*b - a - c/T**2) - \
            k_pres*0.45/T*k_con*k_exp
    return np.hstack([dkdk0, dkdT])

In [ ]:
nn = 10
k0 = np.ones(nn)*3.0
T  = np.linspace(298.0, 1000.0, nn)

dk0 = k0*0.1
dT  = T*0.1

coords = np.linspace(0.0, 120e3, nn)

# constants
rho = 2700.
alpha = 3.5e-5
beta = 7.69e-12
gmma_T = 1.25
K_0p = 4.3
K_T = 130.0
g_av = 9.5


x = np.hstack([k0, T])
dx = np.hstack([dk0, dT])


fm0 = forward_model(x)
fm1 = forward_model(x + dx)
tl = tangent_linear(x, dx)
ad = adjoint_model(x)

print("finite differences = {}".format(fm1-fm0))
print("tangent linear = {}".format(tl))
print("adjoint model = {}".format(ad.dot(dx)))